In [4]:
import math
import pandas as pd
from Bio.PDB import PDBParser
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [5]:
# Function to compute vector between two points
def vector_between(atom1, atom2):
    return [atom2[i] - atom1[i] for i in range(3)]

# Function to compute the cross product of two vectors
def cross_product(v1, v2):
    return [
        v1[1] * v2[2] - v1[2] * v2[1],
        v1[2] * v2[0] - v1[0] * v2[2],
        v1[0] * v2[1] - v1[1] * v2[0],
    ]

# Function to compute the dot product of two vectors
def dot_product(v1, v2):
    return sum(v1[i] * v2[i] for i in range(3))

# Function to calculate the magnitude of a vector
def magnitude(v):
    return math.sqrt(sum(v[i] ** 2 for i in range(3)))

# Function to calculate the angle between two vectors
def angle_between_vectors(v1, v2):
    dot_prod = dot_product(v1, v2)
    mag_v1 = magnitude(v1)
    mag_v2 = magnitude(v2)
    cos_theta = dot_prod / (mag_v1 * mag_v2)
    # Ensure cos_theta is within the valid range for acos
    cos_theta = max(min(cos_theta, 1), -1)
    return math.degrees(math.acos(cos_theta))

# Function to get the coordinates of an atom from its identifier (chain, residue number, atom name)
def get_atom_coordinates(structure, chain_id, residue_number, atom_name):
    chain = structure[0][chain_id]
    residue = chain[residue_number]
    atom = residue[atom_name]
    return atom.coord

# Function to calculate the angle between two planes
def calculate_angle_between_planes(structure, atoms_plane1, atoms_plane2):
    coords1 = [get_atom_coordinates(structure, *atom_id) for atom_id in atoms_plane1]
    coords2 = [get_atom_coordinates(structure, *atom_id) for atom_id in atoms_plane2]

    v1_plane1 = vector_between(coords1[0], coords1[1])
    v2_plane1 = vector_between(coords1[0], coords1[2])
    v1_plane2 = vector_between(coords2[0], coords2[1])
    v2_plane2 = vector_between(coords2[0], coords2[2])

    normal1 = cross_product(v1_plane1, v2_plane1)
    normal2 = cross_product(v1_plane2, v2_plane2)

    angle = angle_between_vectors(normal1, normal2)
    return angle

# Function to find all CYC instances in the PDB file
def find_cyc_instances(structure):
    cyc_instances = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.get_resname() == "CYC":
                    cyc_instances.append((chain.id, residue.id))  # Store chain and residue identifier
    return cyc_instances


In [10]:
# Function to find a residue, accounting for heteroatom flag and insertion code
def find_residue(chain, res_num):
    for res in chain:
        if res.id[1] == res_num:  # Match by residue number
            return res
    raise KeyError(f'Residue {res_num} not found in chain {chain.id}')

# Function to calculate the angle between two planes
def calculate_angle_between_planes(structure, atoms_plane1, atoms_plane2):
    coords1 = [get_atom_coordinates(structure, *atom_id) for atom_id in atoms_plane1]
    coords2 = [get_atom_coordinates(structure, *atom_id) for atom_id in atoms_plane2]
    
    normal1 = np.cross(coords1[1] - coords1[0], coords1[2] - coords1[0])
    normal2 = np.cross(coords2[1] - coords2[0], coords2[2] - coords2[0])
    
    cos_angle = np.dot(normal1, normal2) / (np.linalg.norm(normal1) * np.linalg.norm(normal2))
    angle = np.degrees(np.arccos(cos_angle))
    return angle

# Function to get atom coordinates
def get_atom_coordinates(structure, chain_id, residue_number, atom_name):
    for model in structure:
        chain = model[chain_id]  # Access the chain
        residue = find_residue(chain, residue_number)  # Use find_residue to access the correct residue
        atom = residue[atom_name]  # Access the atom by its name
        return atom.coord  # Return the atom coordinates
    
def recenter_angle(angle):
    if angle < 90:
        return angle  # Keep angles below 90 as is
    else:
        return 180 - angle  # Recenter angles between 90 and 180

# Function to find a residue, accounting for heteroatom flag and insertion code
def find_residue(chain, res_num):
    for res in chain:
        if res.id[1] == res_num:  # Match by residue number
            return res
    raise KeyError(f'Residue {res_num} not found in chain {chain.id}')

# Function to calculate the angle between two planes
def calculate_angle_between_planes(structure, atoms_plane1, atoms_plane2):
    coords1 = [get_atom_coordinates(structure, *atom_id) for atom_id in atoms_plane1]
    coords2 = [get_atom_coordinates(structure, *atom_id) for atom_id in atoms_plane2]
    
    normal1 = np.cross(coords1[1] - coords1[0], coords1[2] - coords1[0])
    normal2 = np.cross(coords2[1] - coords2[0], coords2[2] - coords2[0])
    
    cos_angle = np.dot(normal1, normal2) / (np.linalg.norm(normal1) * np.linalg.norm(normal2))
    angle = np.degrees(np.arccos(cos_angle))
    return angle

# Function to get atom coordinates
def get_atom_coordinates(structure, chain_id, residue_number, atom_name):
    for model in structure:
        chain = model[chain_id]  # Access the chain
        residue = find_residue(chain, residue_number)  # Use find_residue to access the correct residue
        atom = residue[atom_name]  # Access the atom by its name
        return atom.coord  # Return the atom coordinates

# DataFrame to store angles
angles_df = pd.DataFrame(columns=['Angle1', 'Angle2', 'Angle3'])
# Apply the recentering function to the angles DataFrame
angles_df['Angle1_recentered'] = angles_df['Angle1'].apply(recenter_angle)
angles_df['Angle2_recentered'] = angles_df['Angle2'].apply(recenter_angle)
angles_df['Angle3_recentered'] = angles_df['Angle3'].apply(recenter_angle)

In [11]:
# Parse the PDB file
parser = PDBParser()
structure = parser.get_structure('APC', 'APC_FR_FINAL_FINAL.pdb')

/Users/giovanniconsoli/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 37099
  warnings.warn(


In [12]:
# DataFrame to store angles
angles_df = pd.DataFrame(columns=['Angle1', 'Angle2', 'Angle3'])

cyc_instances = find_cyc_instances(structure)
# Loop over each CYC instance and calculate angles
for cyc in cyc_instances:
    chain_id, residue_id = cyc
    residue_num = residue_id[1]  # Corrected to access the middle element
    
    # Define the atoms for the angle calculations
    atoms_plane2 = [(chain_id, residue_num, 'NB'), (chain_id, residue_num, 'C1B'), (chain_id, residue_num, 'C2B')]
    atoms_plane1 = [(chain_id, residue_num, 'NA'), (chain_id, residue_num, 'C3A'), (chain_id, residue_num, 'C4A')]
    atoms_plane4 = [(chain_id, residue_num, 'NA'), (chain_id, residue_num, 'C1A'), (chain_id, residue_num, 'C2A')]
    atoms_plane3 = [(chain_id, residue_num, 'ND'), (chain_id, residue_num, 'C4D'), (chain_id, residue_num, 'C3D')]
    atoms_plane6 = [(chain_id, residue_num, 'ND'), (chain_id, residue_num, 'C2D'), (chain_id, residue_num, 'C1D')]
    atoms_plane5 = [(chain_id, residue_num, 'NC'), (chain_id, residue_num, 'C3C'), (chain_id, residue_num, 'C4C')]

    # Calculate angles
    try:
        angle1 = calculate_angle_between_planes(structure, atoms_plane1, atoms_plane2)  # Angle between plane 1 and plane 2
        angle2 = calculate_angle_between_planes(structure, atoms_plane3, atoms_plane4)  # Angle between plane 3 and plane 4
        angle3 = calculate_angle_between_planes(structure, atoms_plane5, atoms_plane6)  # Angle between plane 5 and plane 6

        # Append angles along with chain and residue info to DataFrame
        new_row = pd.DataFrame({
            'Chain_ID': [chain_id], 
            'Residue_Number': [residue_num], 
            'Angle1': [angle1], 
            'Angle2': [angle2], 
            'Angle3': [angle3]
        })
        angles_df = pd.concat([angles_df, new_row], ignore_index=True)
    except KeyError as e:
        print(f"Error accessing atoms for CYC {cyc}: {e}")

# Print the angles DataFrame
print(angles_df)

        Angle1      Angle2      Angle3 Chain_ID  Residue_Number
0   154.543274  169.151794   20.656744        A           201.0
1   147.611069  170.103256   41.449059        B           201.0
2   150.491409  160.871109   16.974989        C           201.0
3   150.714828  174.181686   32.967144        D           201.0
4    12.603683  158.102829   19.804010        E           201.0
5   130.787964  162.629608   36.591125        F           201.0
6   169.276611  162.298386   27.544197        G           201.0
7   162.220779  168.533112   38.386856        H           201.0
8   169.160416  167.517502   25.779232        I           201.0
9   148.512207  162.011429  175.637436        J           201.0
10   12.238072  163.614471  173.632629        K           801.0
11   56.077045  168.513626   30.667364        L           201.0
12  178.326004  172.074921   21.571045        M           201.0
13   68.289658  149.556030   41.011700        N           201.0
14  171.106537  175.586716   24.535641  

/var/folders/qn/2_xjlk0x073ft9zqwbzn9qdw0000gn/T/ipykernel_90451/1819755564.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  angles_df = pd.concat([angles_df, new_row], ignore_index=True)
